In [1]:
import cobra
import pandas as pd
import json
from collections import OrderedDict
from custom_functions_scRBA import *

#### Build model containing constant coefficients only

In [2]:
model = cobra.Model('SCResAlloModel')

Using license file /home/hvdinh16/Workspace/Softwares/gurobi910/linux64/gurobi.lic
Academic license - for non-commercial use only - expires 2021-07-26


Metabolites

In [3]:
with open('../scRBA_new/species.txt') as f:
    species = f.read().split('\n')
for m in species:
    met = cobra.Metabolite(m)
    model.add_metabolites(met)

Reactions

In [4]:
# Add reactions
df_rxns = pd.read_excel('../scRBA_new/scRAM_network_frompy.xlsx')
df_rxns.index = df_rxns.id.to_list()

In [5]:
for r in df_rxns.index:
    x = metabolites_dict_from_reaction_equation_RBA(df_rxns.reaction[r])
    met_dict = dict()
    for k,v in x.items():
        if is_number(v):
            met_dict[k] = v
    
    rxn = cobra.Reaction(r)
    model.add_reaction(rxn)
    rxn.reaction = build_reaction_equation_from_metabolites_dict_RBA(met_dict, arrow='-->')

Set objective

No objective

In [6]:
model.objective = dict()
cobra.io.save_json_model(model, './models/cobraModel_constantStoichOnly_noObj.json')

#### Build dictionary containing symbolic coefficients

In [7]:
df_rxns = pd.read_excel('../scRBA_new/scRAM_network_frompy.xlsx')
df_rxns.index = df_rxns.id.to_list()
df_rxns = df_rxns[df_rxns.symbolic_coeffs.isnull() == False]

df_keff = pd.read_excel('../scRBA_new/keff_frompy.xlsx')
df_keff.index = df_keff.rxn_id.to_list()

df_pro = pd.read_excel('../scRBA/curation/prot_stoich_curation_2021-05-21.xlsx')
df_pro.index = df_pro.id.to_list()

In [8]:
df_params = pd.read_excel('../scRBA/input/params_respiration.xlsx')
df_params.index = df_params.param_id.to_list()

In [9]:
stoich = OrderedDict()

Symbolic part

In [10]:
kribo = 10.5*3600; dummyNAA = 405;

In [11]:
for i in df_rxns.index:
    met_dict = metabolites_dict_from_reaction_equation_RBA(df_rxns.reaction[i])
    met_dict = {k:v for k,v in met_dict.items() if k != '' and is_number(v) == False}
    sym = df_rxns.symbolic_coeffs[i]
    
    for k,v in met_dict.items():
        if v == '-mu/keff':
            v = str(round(-1 / df_keff.keff[i] / 3600, 12)) + '*mu'
        elif v == '-mu*NAA/kribo':
            pro_id = i[7:]
            if pro_id in ['DUMMYPRO_c', 'DUMMYPRO_m']:
                v = str(-round(dummyNAA / kribo, 12)) + '*mu'
            else:
                v = str(-round((len(df_pro.sequence[pro_id]) - 1) / kribo, 12)) + '*mu'
        else:
            v = df_params.string[v]
            
        stoich[i+' | '+k] = str(v)

In [12]:
stoich['BIODIL-BIODILRESPIRATION' + ' | ' + 'BIO-unmodeledProtBiom'] = '-(0.118970 + 0.139017*mu)'
stoich['BIODIL-BIODILRESPIRATION' + ' | ' + 'BIO-modeledProtBiom_c'] = '-(0.121007 + 0.141397*mu)'
stoich['BIODIL-BIODILRESPIRATION' + ' | ' + 'BIO-modeledProtBiom_m'] = '-(0.131677 + 0.153865*mu)'

stoich['BIODIL-BIODILFERMENTATION' + ' | ' + 'BIO-unmodeledProtBiom'] = '-(0.118970 + 0.139017*mu)'
stoich['BIODIL-BIODILFERMENTATION' + ' | ' + 'BIO-modeledProtBiom_c'] = '-(0.234957 + 0.274546*mu)'
stoich['BIODIL-BIODILFERMENTATION' + ' | ' + 'BIO-modeledProtBiom_m'] = '-(0.017728 + 0.020715*mu)'

Save<br>
For mu < 0.211

In [13]:
import json
from copy import deepcopy

In [14]:
with open('./models/symbolicStoich_lower_0_211.json', 'w') as f: 
    json.dump(stoich, f)

For mu > 0.211

In [15]:
stoich_higher = deepcopy(stoich)
stoich_higher['BIODIL-BIODILRESPIRATION' + ' | ' + 'BIO-unmodeledProtBiom'] = -0.148303012202
stoich_higher['BIODIL-BIODILRESPIRATION' + ' | ' + 'BIO-modeledProtBiom_c'] = -0.150841828568
stoich_higher['BIODIL-BIODILRESPIRATION' + ' | ' + 'BIO-modeledProtBiom_m'] = -0.164142817229

stoich_higher['BIODIL-BIODILFERMENTATION'+ ' | ' + 'BIO-unmodeledProtBiom'] = -0.148303012202
stoich_higher['BIODIL-BIODILFERMENTATION'+ ' | ' + 'BIO-modeledProtBiom_c'] = -0.292885824511
stoich_higher['BIODIL-BIODILFERMENTATION'+ ' | ' + 'BIO-modeledProtBiom_m'] = -0.022098821287

stoich_higher['BIODIL-BIODILRESPIRATION' + ' | ' + 'MET-mannan_c'] = -0.539010001587
stoich_higher['BIODIL-BIODILRESPIRATION' + ' | ' + 'MET-13BDglucan_en'] = -0.836427660720
stoich_higher['BIODIL-BIODILRESPIRATION' + ' | ' + 'MET-16BDglucan_en'] = -0.220112542295
stoich_higher['BIODIL-BIODILRESPIRATION' + ' | ' + 'MET-chtn_c'] = -0.019775196008
stoich_higher['BIODIL-BIODILRESPIRATION' + ' | ' + 'MET-glycogen_c'] = -0.403253827394
stoich_higher['BIODIL-BIODILRESPIRATION' + ' | ' + 'MET-tre_c'] = -0.020213206243
stoich_higher['BIODIL-BIODILRESPIRATION' + ' | ' + 'BIO-rna_c'] = -0.07219476

stoich_higher['BIODIL-BIODILFERMENTATION' + ' | ' + 'MET-mannan_c'] = -0.539010001587
stoich_higher['BIODIL-BIODILFERMENTATION' + ' | ' + 'MET-13BDglucan_en'] = -0.836427660720
stoich_higher['BIODIL-BIODILFERMENTATION' + ' | ' + 'MET-16BDglucan_en'] = -0.220112542295
stoich_higher['BIODIL-BIODILFERMENTATION' + ' | ' + 'MET-chtn_c'] = -0.019775196008
stoich_higher['BIODIL-BIODILFERMENTATION' + ' | ' + 'MET-glycogen_c'] = -0.403253827394
stoich_higher['BIODIL-BIODILFERMENTATION' + ' | ' + 'MET-tre_c'] = -0.020213206243
stoich_higher['BIODIL-BIODILFERMENTATION' + ' | ' + 'BIO-rna_c'] = -0.07219476

In [16]:
with open('./models/symbolicStoich_higher_0_211.json', 'w') as f: 
    json.dump(stoich_higher, f)